In [9]:
from new_model_in_jax import PosteriorgramModel
import haiku as hk
import jax

def f(x):
    a = PosteriorgramModel()
    return a(x)
model = hk.with_empty_state(hk.transform(f))
rng = jax.random.PRNGKey(0)
params, state = model.init(rng, "test.m4a")
model.apply(params, state=state, rng=rng, x="test.m4a")

/home/noahadhikari/.local/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


TypeError: __call__() missing 1 required positional argument: 'state'